**今回の実習内容**
 * ビニング
 * フーリエクロップ
 * パディング
 * 実習課題

In [ ]:
import sys,math,cmath,os,copy
import numpy as np
from numpy import fft
from matplotlib import pyplot as plt
import tifffile as tiff
import mrcfile as mrc

# OpenCV
import cv2

In [ ]:
# テストデータの読み込み
fn_yoshi = "./data/course4_yoshizawa.tif"
yoshi = tiff.imread(fn_yoshi)

# オリジナルの画像サイズ(とその半分)
orisize = yoshi.shape[0]
halfori = int(orisize / 2)
print("input image size: ", orisize)

fig, ax = plt.subplots()

ax.imshow(yoshi, cmap='Greys')

**ビニングとフーリエクロップを比較してみる**
 * 実空間像と逆空間像、それぞれで違いを見る

In [ ]:
# 実空間でビニング
binfactor = 2

# ビニング後のサイズ
binsize = int(orisize / binfactor)

# ビニング後の配列を初期化
yoshi_bin = np.zeros((binsize,binsize))

# ビニング
for i in range(orisize):
    ibin = math.floor(i/binfactor)
    for j in range(orisize):
        jbin = math.floor(j/binfactor)

        yoshi_bin[ibin,jbin] += yoshi[i,j]

yoshi_bin /= binfactor

# ビニング前と比較(左がビニング後)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

ax[0].imshow(yoshi_bin, cmap='Greys')
ax[1].imshow(yoshi, cmap='Greys')

In [ ]:
# 逆空間でのフーリエクロップ
# cryosparcと同様にクロップ後のサイズを指定
cropsize = 150   # should be even
halfcrop = int(cropsize / 2)

# 逆空間像を計算
rec_yoshi = fft.fft2(yoshi)

# クロップ後の逆空間像用の配列を初期化
rec_yoshi_crop = np.zeros((cropsize,cropsize),dtype=complex)

# クロップ
# np.fftでは逆空間中心が四隅になっていることに注意
rec_yoshi_crop[0:halfcrop,0:halfcrop] = rec_yoshi[0:halfcrop,0:halfcrop]               # 左上
rec_yoshi_crop[0:halfcrop,halfcrop:]  = rec_yoshi[0:halfcrop,orisize-halfcrop:]        # 右上
rec_yoshi_crop[halfcrop:,0:halfcrop]  = rec_yoshi[orisize-halfcrop:,0:halfcrop]        # 左下
rec_yoshi_crop[halfcrop:,halfcrop:]   = rec_yoshi[orisize-halfcrop:,orisize-halfcrop:] # 右下

# 逆フーリエ変換で実空間に戻す
yoshi_crop = fft.ifft2(rec_yoshi_crop).real

# クロップ前と比較(左がクロップ後)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

ax[0].imshow(yoshi_crop, cmap='Greys')
ax[1].imshow(yoshi, cmap='Greys')

In [ ]:
# ビニング像とクロップ像を比較(実空間)

std_bin = np.std(yoshi_bin)
std_crop = np.std(yoshi_crop)

yoshi_crop /= (std_crop / std_bin)

diff_real = yoshi_bin - yoshi_crop

# ビニング像、クロップ像、差分
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12,4))

# 画素値の最大、最小が微妙に違うのでコントラストの付け方を統一
ax[0].imshow(yoshi_bin, cmap='Greys', vmin=-1.218, vmax=5.388)
ax[1].imshow(yoshi_crop, cmap='Greys', vmin=-1.218, vmax=5.388)
ax[2].imshow(diff_real, cmap='Greys', vmin=-1.218, vmax=5.388)

**パディング**
 * パディングの関数を定義
 * bガラクトシダーゼのスペックルを見てみる
 * 回転補間の精度を見てみる

In [ ]:
# パディングの関数を定義しておく
def padding(image, factor):
    # paddingのサイズを定義
    padding_size = np.array([int(image.shape[0]*(factor-1)),int(image.shape[1]*(factor-1))])
    margin = np.array([int(padding_size[0]/2),int(padding_size[1]/2)])
    return cv2.copyMakeBorder(image, margin[1], margin[1], margin[0], margin[0], cv2.BORDER_CONSTANT, 0)

In [ ]:
# まずはbgalの画像を読み込み
fn_bgal = "./data/course5_bgal.tif"

# bgalの投影像を読み込み
bgal = tiff.imread(fn_bgal)
print("image size of bgal: ", bgal.shape[0], " x ", bgal.shape[1])

# 先ほど定義した関数でpadding imageも作る
# 2倍にパディング
bgal_pad = padding(bgal, 2)
print("image size of padded bgal: ", bgal_pad.shape[0], " x ", bgal_pad.shape[1])

print(np.max(bgal),np.min(bgal))
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

ax[0].imshow(bgal, cmap='Greys')
ax[1].imshow(bgal_pad, cmap='Greys')

In [ ]:
# 逆空間像のスペックルを比較してみる
rec_bgal = fft.fftshift(fft.fft2(bgal))
rec_bgal_pad = fft.fftshift(fft.fft2(bgal_pad))

ampl_bgal = np.abs(rec_bgal)
ampl_bgal_pad = np.abs(rec_bgal_pad)

second_min = np.unique(ampl_bgal)[1]
for i in range(rec_bgal.shape[0]):
    for j in range(rec_bgal.shape[1]):
        val = ampl_bgal[i,j]
        if val < second_min:
            ampl_bgal[i,j] = np.log(second_min)
        else:
            ampl_bgal[i,j] = np.log(ampl_bgal[i,j])

second_min = np.unique(ampl_bgal_pad)[1]
for i in range(rec_bgal_pad.shape[0]):
    for j in range(rec_bgal_pad.shape[1]):
        val = ampl_bgal_pad[i,j]
        if val < second_min:
            ampl_bgal_pad[i,j] = np.log(second_min)
        else:
            ampl_bgal_pad[i,j] = np.log(ampl_bgal_pad[i,j])

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

ax[0].imshow(ampl_bgal, cmap='viridis')
ax[1].imshow(ampl_bgal_pad, cmap='viridis') # パディングしても表示される逆空間像の範囲は同じ

In [ ]:
# よくわからんので逆空間像を拡大して比較

center = [int(rec_bgal.shape[0]/2),int(rec_bgal.shape[1]/2)]
print(type(rec_bgal[0,0]))

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

# 画像左上が逆空間中心
ax[0].imshow(ampl_bgal[center[0]:center[0]+40,center[1]:center[1]+40], cmap='viridis')
ax[1].imshow(ampl_bgal_pad[2*center[0]:2*(center[0]+40),2*center[1]:2*(center[1]+40)], cmap='viridis')

**実習課題**

**本当はこの後逆空間像の回転をして、パディングの影響を見るつもりでした**

**なぜか回転するとよくわからないことが起きたので一旦別の課題にします**
 * 以下の2枚の画像を作ってください
 * 1枚目: course5_bgal.tifを90度回転させた像
 * 2枚目: course5_bgal.tifを10度ずつ9回、計90度回転させた像
 * 像の回転は02_correlationで使ったcv2.getRotationMatrix2Dとcv2.warpAffineを使ってください
 * 補完は全てLINEARを使ってください
 * 最後に、2枚の像のFRC(FSCの二次元版)を計算してください
 * 下の関数が使えますが、「画像中心が逆空間の原点になっている逆空間像」を入力してください。
 * frc = calcFRC(rec1,rec2)

In [ ]:
# input: 逆空間像rec1とrec2(complexのnp.ndarray)
# また、rec1, rec2は画像中心が逆空間原点となっていると仮定
# output: 空間周波数とFRCを格納した二次元配列
# 1列目: 空間周波数、2列目: FRC
def calcFRC(rec1,rec2):
    size = rec1.shape[0]
    center = np.array([(size/2)-0.5,(size/2)-0.5])

    frc = np.zeros((size/2,2))
    den1 = np.zeros(size/2)
    den2 = np.zeros(size/2)

    for i in range(rec1.shape[0]):
        kx = float(i) - center[0]
        for j in range(rec1.shape[0]):
            ky = float(j) - center[1]

            idx = int(np.sqrt(kx*kx + ky*ky))
            z1 = rec1[i,j]
            z2 = rec2[i,j]
            absz1 = np.abs(z1)
            absz2 = np.abs(z2)

            frc[idx] += (z1.conjugate() * z2).real        
            den1[idx] += absz1*absz1
            den2[idx] += absz2*absz2

    for i in range(frc.shape[0]):
        frc[i,0] = (float(i) + 0.5) / size
        frc[i,1] /= np.sqrt(den1[i]*den2[i])

    return copy.deepcopy(frc)